# Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.

**Задача:** поиск токсичных коментариев для отправки на модерацию.

**Цель:** обучить модель классифицировать комментарии на позитивные и негативные на наборе данных с разметкой о токсичности. Метрика f1 модели должна иметь значение не меньше 0,75.

In [72]:
import pandas as pd
import numpy as np
import re
import spacy
from scipy.sparse import vstack
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tqdm._tqdm_notebook import tqdm_notebook
from catboost import CatBoostClassifier
tqdm_notebook.pandas()

import warnings
warnings.filterwarnings(action='ignore')

# 1 Загрузка и подготовка данных

## 1.1 Загрузка данных

In [2]:
data = pd.read_csv('datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные загружены, пропусков нет. Требуется обработка текстов.

## 1.2 Преобразование текста

**Лемматизация и замена знаков**

Прежде всего нужно привести слова к их основной форме. Используем библиотеку spacy для ускорения процесса. Для замены не буквенных символов воспользуемся регулярными выражениями

In [5]:
nlp = spacy.load('en_core_web_sm')

# определим функцию для лемматизации текстов
def lemmatize_text(row, nlp=nlp):
    doc = nlp(row)
    
    # лемматизируем
    lemma = ' '.join([token.lemma_ for token in doc]).lower()
    
    # убираем знаки, лишние пробелы и возвращаем результат
    return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', lemma).split())

Применим функцию ко всем строкам в data['text'], проконтролируем прогресс с помощью tqdm. Так как процесс лемматизации очень длительный - применим следующую конструкцию, которая позволит не проводить лемматизацию при каждом перезапуске jupyter.

In [6]:
try:
    data_lemm = pd.read_csv('datasets/data_lemm.csv')
    print('Данные загружены!')
    del data
except FileNotFoundError:
    print('Создадим и сохраним лемматизированные тексты в файл:')
    data_lemm = data.copy()
    del data
    data_lemm['text'] = data_lemm['text'].progress_apply(lemmatize_text)   # отражаем прогресс-бар при выполнении apply
    data_lemm.to_csv('datasets/data_lemm.csv', index=False)
    print('Данные сохранены.')

Данные загружены!


In [7]:
data_lemm.head()

,toxic,text
0,0,explanation why the edit make under pron usern...
1,0,d'aww pron match this background colour pron b...
2,0,hey man pron be really not try to edit war pro...
3,0,more pron can not make any real suggestion on ...
4,0,pron sir be pron hero any chance pron remember...


**TF-IDF для корпуса текстов**

Подгрузим стоп-слова:

In [8]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(stopwords)

{'n‘t', 'forty', '‘m', 'well', 'just', 'around', 'in', "'s", 'until', 'whereafter', 'such', 'among', 'does', 'sometime', 'himself', 'his', 'towards', 'meanwhile', 'against', 'mostly', 'hereby', 'fifteen', 'unless', 'few', 'ours', 'really', 'something', 'most', 'or', 'they', 'bottom', 'of', 'sixty', 'six', 'and', 'on', 'none', 'whenever', '‘d', 'why', '’re', 'him', 'never', 'whom', '’d', 'perhaps', 'might', 'almost', '’s', 'nothing', 'see', 'this', 'eleven', 'nobody', 'became', 'my', 'was', 'how', 'whereby', 'less', 'than', 'beside', 'third', 'else', 'where', 'did', 'during', 'their', 'i', 're', "'ll", 'any', 'below', 'alone', 'themselves', 'them', 'out', 'anyhow', 'anyone', 'except', 'yourself', 'two', 'anyway', 'namely', 'if', 'first', 'cannot', 'made', 'whence', 'beforehand', 'whatever', 'now', 'although', 'top', 'move', 'besides', 'been', 'everyone', 'side', 'therefore', 'four', 'either', 'take', 'down', 'are', 'everything', 'without', 'otherwise', 'thence', 'about', 'give', 'you', 

Создадим счетчик-преобразователь в вектора слов с учетом стоп-слов:

In [9]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Для дальнейших преобразований нужно для начала разбить признаки на обучающие и тестовые. Валидацию модели будем проводить на кроссвалидации.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_lemm.drop('toxic', axis=1), data_lemm['toxic'], random_state=42, test_size=0.25)

Создадим вектора из набора текстов для обучающей и тестовой выборки:

In [11]:
X_train_vec = count_tf_idf.fit_transform(X_train.values.astype('U').ravel())

In [12]:
X_train_vec.shape

(119678, 133016)

На основе обученного счетчика создадим признаки для тестовых данных

In [13]:
X_test_vec = count_tf_idf.transform(X_test.values.astype('U').ravel())

In [63]:
X_test_vec.shape

(39893, 133016)

### Вывод:

1. Для набора данных была проведена лемматизация текстов с помощью библиотеки spacy. Эта библиотека выигрывает в скорости у pymystem3 и имеет все необходимое для решения задачи.
2. Произведена очистка текстов с помощью регулярных выражений.
2. Датасет разделен на обучающую и тестовую выборки, проведена процедура векторизации признаков TF-IDF, для полученных наборов и размеров выборок количество признаков равно 133016. При изменении размеров тестовой выборки это значение будет меняться.

# 2 Обучение моделей

Для выполнения задачи классификации используем модели логистической регрессии, бустер Catboost и случайную модель, на которой проверим адекватность выбраной модели.

## 2.1 Обучения моделей с гиперпараметрами по умолчанию

In [15]:
# создаем объект случайной модели
model_dummy = DummyClassifier()

# создаем объект модели логистической регрессии
model_log = LogisticRegression(random_state=42)

# создаем модель Catboost (выберем число итераций = 10 для ускорения процесса кроссвалидации)
model_cat = CatBoostClassifier(random_state=42, iterations=10)

Напишем функцию, с помощью которой можно будт оценивать и выводить на экран показатели модели:

In [16]:
def train_model(model, name):
    print('*' * 45)
    print(f'Классификация текстов для модели {name}')
    score = cross_val_score(model, X_train_vec, y_train, cv=3, scoring='f1')
    print(f'Средняя оценка метрики f1 на кроссвалидации => {score.mean()}')
    print()
    print('*' * 45)

In [17]:
for i in ((model_dummy, 'Случайная модель'), (model_log, 'Логистическая регрессия'), (model_cat, 'Catboost')):
    train_model(i[0], i[1])

*********************************************
Классификация текстов для модели Случайная модель
Средняя оценка метрики f1 на кроссвалидации => 0.10299318699638256

*********************************************
*********************************************
Классификация текстов для модели Логистическая регрессия
Средняя оценка метрики f1 на кроссвалидации => 0.7024755655896494

*********************************************
*********************************************
Классификация текстов для модели Catboost
Learning rate set to 0.5
0:	learn: 0.3442647	total: 1.21s	remaining: 10.9s
1:	learn: 0.2572535	total: 2.33s	remaining: 9.34s
2:	learn: 0.2314581	total: 3.43s	remaining: 8.01s
3:	learn: 0.2164409	total: 4.51s	remaining: 6.76s
4:	learn: 0.2081470	total: 5.61s	remaining: 5.61s
5:	learn: 0.2015730	total: 6.73s	remaining: 4.49s
6:	learn: 0.1965784	total: 7.84s	remaining: 3.36s
7:	learn: 0.1918447	total: 8.94s	remaining: 2.23s
8:	learn: 0.1881516	total: 10.1s	remaining: 1.12s
9:	learn: 0

### Вывод:

Лучшая модель на кроссвалидации с гиперпараметрами по умолчанию - модель Логистической регрессии (f1 = 0.7). Случайная модель очень плохая.

## 2.2 Оптимизация моделей

В исходных данных есть огромный дисбаланс классов:

In [18]:
y_train.value_counts()

0    107512
1     12166
Name: toxic, dtype: int64

Перевес в 8-9 раз. Добавим параметр class_weight в модель логистической регрессии и модель catboost:

In [19]:
model_log_2 = LogisticRegression(random_state=42, class_weight='balanced')

In [20]:
%%time
train_model(model_log_2, 'Логистическая регрессия (баланс)')

*********************************************
Классификация текстов для модели Логистическая регрессия (баланс)
Средняя оценка метрики f1 на кроссвалидации => 0.7454919368376918

*********************************************
Wall time: 5.34 s


In [21]:
model_cat_2 = CatBoostClassifier(random_state=42, iterations=10, class_weights=[1, np.ceil(y_train.value_counts()[0] / y_train.value_counts()[1])])

In [22]:
%%time
train_model(model_cat_2, 'Catboost (баланс)')

*********************************************
Классификация текстов для модели Catboost (баланс)
Learning rate set to 0.5
0:	learn: 0.5617049	total: 1.13s	remaining: 10.2s
1:	learn: 0.5115736	total: 2.25s	remaining: 9s
2:	learn: 0.4815795	total: 3.3s	remaining: 7.69s
3:	learn: 0.4648492	total: 4.36s	remaining: 6.54s
4:	learn: 0.4524839	total: 5.46s	remaining: 5.46s
5:	learn: 0.4408418	total: 6.54s	remaining: 4.36s
6:	learn: 0.4278960	total: 7.6s	remaining: 3.25s
7:	learn: 0.4148006	total: 8.67s	remaining: 2.17s
8:	learn: 0.4073013	total: 9.79s	remaining: 1.09s
9:	learn: 0.4015988	total: 10.9s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5760993	total: 1.15s	remaining: 10.4s
1:	learn: 0.5139515	total: 2.19s	remaining: 8.77s
2:	learn: 0.4861711	total: 3.32s	remaining: 7.74s
3:	learn: 0.4684976	total: 4.43s	remaining: 6.65s
4:	learn: 0.4540945	total: 5.55s	remaining: 5.55s
5:	learn: 0.4401429	total: 6.66s	remaining: 4.44s
6:	learn: 0.4308809	total: 7.83s	remaining: 3.36s
7:	learn: 

Модель логистической регрессии лучше, но не достаточно (по крайней мере на кросс-валидации). Попробуем поперебирать гиперпараметры обоих моделей.

## 2.3 Подбор гиперпараметров

In [23]:
params_log = {
    'penalty': ['l1', 'l2'],
    'C': range(4, 6)
}

params_cat = {
    'iterations': [500, 1000]
}

Создадим объекты решетчатого поиска:

In [24]:
grid_log = GridSearchCV(model_log_2, params_log, cv=3, verbose=5, scoring='f1')

In [25]:
grid_cat = GridSearchCV(model_cat_2, params_cat, cv=3, verbose=5, scoring='f1')

Обучим модель логистической регрессии:

In [26]:
grid_log.fit(X_train_vec, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=4, penalty=l1 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..................... C=4, penalty=l1, score=0.753, total=   2.2s
[CV] C=4, penalty=l1 .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] ..................... C=4, penalty=l1, score=0.769, total=   2.3s
[CV] C=4, penalty=l1 .................................................


...

[CV] ..................... C=4, penalty=l2, score=0.754, total=   2.4s
[CV] C=4, penalty=l2 .................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    9.0s remaining:    0.0s


[CV] ..................... C=4, penalty=l2, score=0.764, total=   2.9s
[CV] C=4, penalty=l2 .................................................
[CV] ..................... C=4, penalty=l2, score=0.761, total=   2.9s
[CV] C=5, penalty=l1 .................................................
[CV] ..................... C=5, penalty=l1, score=0.753, total=   2.7s
[CV] C=5, penalty=l1 .................................................
[CV] ..................... C=5, penalty=l1, score=0.768, total=   2.2s
[CV] C=5, penalty=l1 .................................................
[CV] ..................... C=5, penalty=l1, score=0.755, total=   2.3s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.753, total=   2.7s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.765, total=   2.8s
[CV] C=5, penalty=l2 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   30.1s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': range(4, 6), 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=5)

In [27]:
print(f'Лучшие параметры логистической регрессии - {grid_log.best_params_}, f1_score = {grid_log.best_score_}')

Лучшие параметры логистической регрессии - {'C': 4, 'penalty': 'l1'}, f1_score = 0.7601577419506556


In [28]:
grid_cat.fit(X_train_vec, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] iterations=500 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Learning rate set to 0.126211
0:	learn: 0.6362751	total: 1.15s	remaining: 9m 32s
1:	learn: 0.6016549	total: 2.45s	remaining: 10m 9s
2:	learn: 0.5820777	total: 3.65s	remaining: 10m 4s
3:	learn: 0.5640260	total: 4.76s	remaining: 9m 50s
4:	learn: 0.5491668	total: 5.82s	remaining: 9m 36s
5:	learn: 0.5377342	total: 6.95s	remaining: 9m 31s
6:	learn: 0.5269031	total: 7.99s	remaining: 9m 22s
7:	learn: 0.5189855	total: 9.09s	remaining: 9m 19s
8:	learn: 0.5123693	total: 10.2s	remaining: 9m 15s
9:	learn: 0.5077169	total: 11.3s	remaining: 9m 14s
10:	learn: 0.5025124	total: 12.4s	remaining: 9m 12s
11:	learn: 0.4951054	total: 13.5s	remaining: 9m 8s
12:	learn: 0.4904826	total: 14.6s	remaining: 9m 6s
13:	learn: 0.4827999	total: 15.7s	remaining: 9m 5s
14:	learn: 0.4760212	total: 16.8s	remaining: 9m 4s
15:	learn: 0.4699841	total: 17.9s	remaining: 9m 2s
16:	learn: 0.4666887	total: 19s	remaining: 9m
17:	learn: 0.4619926	total: 20.2s	remaining: 8m 59s
18:	learn: 0.4583475	total: 21.2s	remaining: 8m 57s
19:

156:	learn: 0.2995849	total: 2m 55s	remaining: 6m 22s
157:	learn: 0.2990171	total: 2m 56s	remaining: 6m 21s
158:	learn: 0.2984864	total: 2m 57s	remaining: 6m 20s
159:	learn: 0.2979781	total: 2m 58s	remaining: 6m 19s
160:	learn: 0.2972459	total: 2m 59s	remaining: 6m 17s
161:	learn: 0.2966701	total: 3m	remaining: 6m 16s
162:	learn: 0.2954423	total: 3m 1s	remaining: 6m 15s
163:	learn: 0.2948613	total: 3m 2s	remaining: 6m 14s
164:	learn: 0.2944928	total: 3m 3s	remaining: 6m 13s
165:	learn: 0.2940773	total: 3m 5s	remaining: 6m 12s
166:	learn: 0.2934495	total: 3m 6s	remaining: 6m 11s
167:	learn: 0.2929685	total: 3m 7s	remaining: 6m 10s
168:	learn: 0.2925516	total: 3m 8s	remaining: 6m 8s
169:	learn: 0.2920343	total: 3m 9s	remaining: 6m 7s
170:	learn: 0.2916409	total: 3m 10s	remaining: 6m 6s
171:	learn: 0.2909926	total: 3m 11s	remaining: 6m 5s
172:	learn: 0.2904815	total: 3m 13s	remaining: 6m 4s
173:	learn: 0.2900400	total: 3m 14s	remaining: 6m 4s
174:	learn: 0.2896326	total: 3m 15s	remaining:

309:	learn: 0.2406652	total: 5m 41s	remaining: 3m 29s
310:	learn: 0.2403833	total: 5m 42s	remaining: 3m 28s
311:	learn: 0.2402093	total: 5m 43s	remaining: 3m 26s
312:	learn: 0.2399553	total: 5m 44s	remaining: 3m 25s
313:	learn: 0.2394302	total: 5m 45s	remaining: 3m 24s
314:	learn: 0.2391364	total: 5m 46s	remaining: 3m 23s
315:	learn: 0.2388813	total: 5m 47s	remaining: 3m 22s
316:	learn: 0.2385821	total: 5m 48s	remaining: 3m 21s
317:	learn: 0.2382709	total: 5m 49s	remaining: 3m 20s
318:	learn: 0.2380635	total: 5m 50s	remaining: 3m 19s
319:	learn: 0.2378327	total: 5m 52s	remaining: 3m 18s
320:	learn: 0.2373964	total: 5m 53s	remaining: 3m 17s
321:	learn: 0.2371836	total: 5m 54s	remaining: 3m 15s
322:	learn: 0.2370510	total: 5m 55s	remaining: 3m 14s
323:	learn: 0.2368608	total: 5m 56s	remaining: 3m 13s
324:	learn: 0.2366144	total: 5m 57s	remaining: 3m 12s
325:	learn: 0.2364876	total: 5m 59s	remaining: 3m 11s
326:	learn: 0.2363129	total: 6m	remaining: 3m 10s
327:	learn: 0.2360153	total: 6m 

...

453:	learn: 0.2581731	total: 11m 58s	remaining: 14m 24s
454:	learn: 0.2579598	total: 12m	remaining: 14m 22s
455:	learn: 0.2578399	total: 12m 1s	remaining: 14m 21s
456:	learn: 0.2576772	total: 12m 3s	remaining: 14m 19s
457:	learn: 0.2574760	total: 12m 5s	remaining: 14m 18s
458:	learn: 0.2572632	total: 12m 6s	remaining: 14m 16s
459:	learn: 0.2571229	total: 12m 8s	remaining: 14m 14s
460:	learn: 0.2569638	total: 12m 9s	remaining: 14m 13s
461:	learn: 0.2568080	total: 12m 11s	remaining: 14m 11s
462:	learn: 0.2566477	total: 12m 13s	remaining: 14m 10s
463:	learn: 0.2564272	total: 12m 15s	remaining: 14m 9s
464:	learn: 0.2562442	total: 12m 17s	remaining: 14m 8s
465:	learn: 0.2560780	total: 12m 18s	remaining: 14m 6s
466:	learn: 0.2559019	total: 12m 20s	remaining: 14m 5s
467:	learn: 0.2557700	total: 12m 22s	remaining: 14m 3s
468:	learn: 0.2555786	total: 12m 24s	remaining: 14m 2s
469:	learn: 0.2554220	total: 12m 25s	remaining: 14m
470:	learn: 0.2553217	total: 12m 27s	remaining: 13m 59s
471:	learn: 

601:	learn: 0.2364913	total: 15m 48s	remaining: 10m 27s
602:	learn: 0.2363783	total: 15m 50s	remaining: 10m 25s
603:	learn: 0.2362447	total: 15m 52s	remaining: 10m 24s
604:	learn: 0.2361721	total: 15m 53s	remaining: 10m 22s
605:	learn: 0.2360553	total: 15m 55s	remaining: 10m 21s
606:	learn: 0.2359946	total: 15m 56s	remaining: 10m 19s
607:	learn: 0.2358964	total: 15m 58s	remaining: 10m 17s
608:	learn: 0.2357998	total: 15m 59s	remaining: 10m 16s
609:	learn: 0.2356757	total: 16m 1s	remaining: 10m 14s
610:	learn: 0.2355241	total: 16m 2s	remaining: 10m 13s
611:	learn: 0.2354153	total: 16m 4s	remaining: 10m 11s
612:	learn: 0.2352916	total: 16m 6s	remaining: 10m 9s
613:	learn: 0.2351151	total: 16m 7s	remaining: 10m 8s
614:	learn: 0.2350195	total: 16m 8s	remaining: 10m 6s
615:	learn: 0.2348905	total: 16m 10s	remaining: 10m 4s
616:	learn: 0.2347677	total: 16m 11s	remaining: 10m 3s
617:	learn: 0.2346949	total: 16m 13s	remaining: 10m 1s
618:	learn: 0.2345756	total: 16m 14s	remaining: 9m 59s
619:	

751:	learn: 0.2191956	total: 19m 41s	remaining: 6m 29s
752:	learn: 0.2190920	total: 19m 43s	remaining: 6m 28s
753:	learn: 0.2190430	total: 19m 44s	remaining: 6m 26s
754:	learn: 0.2189785	total: 19m 46s	remaining: 6m 25s
755:	learn: 0.2189310	total: 19m 48s	remaining: 6m 23s
756:	learn: 0.2188307	total: 19m 49s	remaining: 6m 21s
757:	learn: 0.2187254	total: 19m 51s	remaining: 6m 20s
758:	learn: 0.2186057	total: 19m 52s	remaining: 6m 18s
759:	learn: 0.2184981	total: 19m 54s	remaining: 6m 17s
760:	learn: 0.2183934	total: 19m 56s	remaining: 6m 15s
761:	learn: 0.2182921	total: 19m 57s	remaining: 6m 14s
762:	learn: 0.2182062	total: 19m 59s	remaining: 6m 12s
763:	learn: 0.2181050	total: 20m 1s	remaining: 6m 11s
764:	learn: 0.2180286	total: 20m 2s	remaining: 6m 9s
765:	learn: 0.2179288	total: 20m 4s	remaining: 6m 7s
766:	learn: 0.2178807	total: 20m 6s	remaining: 6m 6s
767:	learn: 0.2177654	total: 20m 7s	remaining: 6m 4s
768:	learn: 0.2176659	total: 20m 9s	remaining: 6m 3s
769:	learn: 0.2175716

902:	learn: 0.2056781	total: 23m 36s	remaining: 2m 32s
903:	learn: 0.2055860	total: 23m 38s	remaining: 2m 30s
904:	learn: 0.2055461	total: 23m 40s	remaining: 2m 29s
905:	learn: 0.2054616	total: 23m 41s	remaining: 2m 27s
906:	learn: 0.2053744	total: 23m 43s	remaining: 2m 25s
907:	learn: 0.2052829	total: 23m 45s	remaining: 2m 24s
908:	learn: 0.2051943	total: 23m 46s	remaining: 2m 22s
909:	learn: 0.2051529	total: 23m 48s	remaining: 2m 21s
910:	learn: 0.2050314	total: 23m 50s	remaining: 2m 19s
911:	learn: 0.2049767	total: 23m 52s	remaining: 2m 18s
912:	learn: 0.2048457	total: 23m 54s	remaining: 2m 16s
913:	learn: 0.2048044	total: 23m 56s	remaining: 2m 15s
914:	learn: 0.2047631	total: 23m 58s	remaining: 2m 13s
915:	learn: 0.2047219	total: 23m 59s	remaining: 2m 12s
916:	learn: 0.2046393	total: 24m 1s	remaining: 2m 10s
917:	learn: 0.2045525	total: 24m 2s	remaining: 2m 8s
918:	learn: 0.2044770	total: 24m 4s	remaining: 2m 7s
919:	learn: 0.2044359	total: 24m 5s	remaining: 2m 5s
920:	learn: 0.204

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostClassifier object at 0x000000002CC075C8>,
             iid='warn', n_jobs=None, param_grid={'iterations': [500, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=5)

In [29]:
print(f'Лучшие параметры случайного леса - {grid_cat.best_params_}, score = {grid_cat.best_score_}')

Лучшие параметры случайного леса - {'iterations': 1000}, score = 0.750951538815851


F1 для обоих моделей больше 0,75, проверим качество предсказаний на тестовой выборке:

In [30]:
def test_validate(model, name):
    print('*' * 45)
    print(f'Проверка качества модели {name} на тестовой выборке')
    model.fit(X_train_vec, y_train)
    predict = model.predict(X_test_vec)
    print(f'f1_score предсказаний => {f1_score(y_test, predict)}')
    print()
    print('*' * 45)

In [31]:
for i in ((grid_log.best_estimator_, 'Логистическая регрессия'), (grid_cat.best_estimator_, 'CatBoost')):
    test_validate(i[0], i[1])

*********************************************
Проверка качества модели Логистическая регрессия на тестовой выборке
f1_score предсказаний => 0.7680860805860805

*********************************************
*********************************************
Проверка качества модели CatBoost на тестовой выборке
Learning rate set to 0.079478
0:	learn: 0.6559167	total: 1.57s	remaining: 26m 14s
1:	learn: 0.6270558	total: 3.09s	remaining: 25m 41s
2:	learn: 0.6071170	total: 4.62s	remaining: 25m 33s
3:	learn: 0.5899061	total: 6.17s	remaining: 25m 37s
4:	learn: 0.5767099	total: 7.71s	remaining: 25m 35s
5:	learn: 0.5645835	total: 9.28s	remaining: 25m 37s
6:	learn: 0.5561485	total: 10.9s	remaining: 25m 49s
7:	learn: 0.5477263	total: 12.5s	remaining: 25m 48s
8:	learn: 0.5411716	total: 14s	remaining: 25m 42s
9:	learn: 0.5354903	total: 15.6s	remaining: 25m 43s
10:	learn: 0.5294459	total: 17.1s	remaining: 25m 39s
11:	learn: 0.5221490	total: 18.7s	remaining: 25m 36s
12:	learn: 0.5167986	total: 20.2s	remain

147:	learn: 0.3516518	total: 3m 52s	remaining: 22m 18s
148:	learn: 0.3511534	total: 3m 54s	remaining: 22m 17s
149:	learn: 0.3506777	total: 3m 55s	remaining: 22m 16s
150:	learn: 0.3501511	total: 3m 57s	remaining: 22m 14s
151:	learn: 0.3497117	total: 3m 58s	remaining: 22m 12s
152:	learn: 0.3492222	total: 4m	remaining: 22m 11s
153:	learn: 0.3486688	total: 4m 2s	remaining: 22m 9s
154:	learn: 0.3479995	total: 4m 3s	remaining: 22m 8s
155:	learn: 0.3473135	total: 4m 5s	remaining: 22m 7s
156:	learn: 0.3467873	total: 4m 6s	remaining: 22m 5s
157:	learn: 0.3462556	total: 4m 8s	remaining: 22m 4s
158:	learn: 0.3458133	total: 4m 10s	remaining: 22m 3s
159:	learn: 0.3446864	total: 4m 11s	remaining: 22m 2s
160:	learn: 0.3442501	total: 4m 13s	remaining: 22m
161:	learn: 0.3434199	total: 4m 14s	remaining: 21m 59s
162:	learn: 0.3429757	total: 4m 16s	remaining: 21m 57s
163:	learn: 0.3424307	total: 4m 18s	remaining: 21m 56s
164:	learn: 0.3419518	total: 4m 19s	remaining: 21m 54s
165:	learn: 0.3414764	total: 4

298:	learn: 0.2919931	total: 7m 54s	remaining: 18m 32s
299:	learn: 0.2917574	total: 7m 55s	remaining: 18m 30s
300:	learn: 0.2915457	total: 7m 57s	remaining: 18m 28s
301:	learn: 0.2912946	total: 7m 59s	remaining: 18m 27s
302:	learn: 0.2910631	total: 8m	remaining: 18m 25s
303:	learn: 0.2907980	total: 8m 2s	remaining: 18m 23s
304:	learn: 0.2905281	total: 8m 3s	remaining: 18m 22s
305:	learn: 0.2903231	total: 8m 5s	remaining: 18m 20s
306:	learn: 0.2900173	total: 8m 7s	remaining: 18m 19s
307:	learn: 0.2896345	total: 8m 8s	remaining: 18m 18s
308:	learn: 0.2894078	total: 8m 10s	remaining: 18m 16s
309:	learn: 0.2891823	total: 8m 11s	remaining: 18m 14s
310:	learn: 0.2889247	total: 8m 13s	remaining: 18m 13s
311:	learn: 0.2886850	total: 8m 14s	remaining: 18m 11s
312:	learn: 0.2883998	total: 8m 16s	remaining: 18m 9s
313:	learn: 0.2881651	total: 8m 18s	remaining: 18m 8s
314:	learn: 0.2879479	total: 8m 19s	remaining: 18m 6s
315:	learn: 0.2876437	total: 8m 21s	remaining: 18m 4s
316:	learn: 0.2874473	t

447:	learn: 0.2597455	total: 11m 54s	remaining: 14m 40s
448:	learn: 0.2595905	total: 11m 56s	remaining: 14m 38s
449:	learn: 0.2594471	total: 11m 57s	remaining: 14m 37s
450:	learn: 0.2593480	total: 11m 59s	remaining: 14m 35s
451:	learn: 0.2592025	total: 12m	remaining: 14m 33s
452:	learn: 0.2588644	total: 12m 2s	remaining: 14m 32s
453:	learn: 0.2581731	total: 12m 3s	remaining: 14m 30s
454:	learn: 0.2579598	total: 12m 5s	remaining: 14m 29s
455:	learn: 0.2578399	total: 12m 7s	remaining: 14m 27s
456:	learn: 0.2576772	total: 12m 8s	remaining: 14m 25s
457:	learn: 0.2574760	total: 12m 10s	remaining: 14m 24s
458:	learn: 0.2572632	total: 12m 11s	remaining: 14m 22s
459:	learn: 0.2571229	total: 12m 13s	remaining: 14m 20s
460:	learn: 0.2569638	total: 12m 15s	remaining: 14m 19s
461:	learn: 0.2568080	total: 12m 16s	remaining: 14m 17s
462:	learn: 0.2566477	total: 12m 18s	remaining: 14m 16s
463:	learn: 0.2564272	total: 12m 19s	remaining: 14m 14s
464:	learn: 0.2562442	total: 12m 21s	remaining: 14m 12s
4

595:	learn: 0.2373585	total: 15m 38s	remaining: 10m 36s
596:	learn: 0.2372555	total: 15m 39s	remaining: 10m 34s
597:	learn: 0.2370404	total: 15m 41s	remaining: 10m 33s
598:	learn: 0.2368592	total: 15m 43s	remaining: 10m 31s
599:	learn: 0.2367058	total: 15m 45s	remaining: 10m 30s
600:	learn: 0.2365764	total: 15m 47s	remaining: 10m 29s
601:	learn: 0.2364913	total: 15m 49s	remaining: 10m 27s
602:	learn: 0.2363783	total: 15m 51s	remaining: 10m 26s
603:	learn: 0.2362447	total: 15m 53s	remaining: 10m 24s
604:	learn: 0.2361721	total: 15m 54s	remaining: 10m 23s
605:	learn: 0.2360553	total: 15m 56s	remaining: 10m 21s
606:	learn: 0.2359946	total: 15m 58s	remaining: 10m 20s
607:	learn: 0.2358964	total: 15m 59s	remaining: 10m 18s
608:	learn: 0.2357998	total: 16m 1s	remaining: 10m 17s
609:	learn: 0.2356757	total: 16m 2s	remaining: 10m 15s
610:	learn: 0.2355241	total: 16m 4s	remaining: 10m 13s
611:	learn: 0.2354153	total: 16m 5s	remaining: 10m 12s
612:	learn: 0.2352916	total: 16m 7s	remaining: 10m 1

745:	learn: 0.2197608	total: 19m 39s	remaining: 6m 41s
746:	learn: 0.2196829	total: 19m 40s	remaining: 6m 39s
747:	learn: 0.2195681	total: 19m 42s	remaining: 6m 38s
748:	learn: 0.2195010	total: 19m 43s	remaining: 6m 36s
749:	learn: 0.2194281	total: 19m 45s	remaining: 6m 35s
750:	learn: 0.2192921	total: 19m 46s	remaining: 6m 33s
751:	learn: 0.2191956	total: 19m 48s	remaining: 6m 31s
752:	learn: 0.2190920	total: 19m 49s	remaining: 6m 30s
753:	learn: 0.2190430	total: 19m 51s	remaining: 6m 28s
754:	learn: 0.2189785	total: 19m 52s	remaining: 6m 26s
755:	learn: 0.2189310	total: 19m 54s	remaining: 6m 25s
756:	learn: 0.2188307	total: 19m 55s	remaining: 6m 23s
757:	learn: 0.2187254	total: 19m 56s	remaining: 6m 22s
758:	learn: 0.2186057	total: 19m 58s	remaining: 6m 20s
759:	learn: 0.2184981	total: 19m 59s	remaining: 6m 18s
760:	learn: 0.2183934	total: 20m 1s	remaining: 6m 17s
761:	learn: 0.2182921	total: 20m 2s	remaining: 6m 15s
762:	learn: 0.2182062	total: 20m 4s	remaining: 6m 14s
763:	learn: 0

896:	learn: 0.2061044	total: 23m 32s	remaining: 2m 42s
897:	learn: 0.2060133	total: 23m 33s	remaining: 2m 40s
898:	learn: 0.2059719	total: 23m 35s	remaining: 2m 39s
899:	learn: 0.2058756	total: 23m 36s	remaining: 2m 37s
900:	learn: 0.2058024	total: 23m 38s	remaining: 2m 35s
901:	learn: 0.2057290	total: 23m 39s	remaining: 2m 34s
902:	learn: 0.2056781	total: 23m 41s	remaining: 2m 32s
903:	learn: 0.2055860	total: 23m 42s	remaining: 2m 31s
904:	learn: 0.2055461	total: 23m 44s	remaining: 2m 29s
905:	learn: 0.2054616	total: 23m 45s	remaining: 2m 27s
906:	learn: 0.2053744	total: 23m 47s	remaining: 2m 26s
907:	learn: 0.2052829	total: 23m 48s	remaining: 2m 24s
908:	learn: 0.2051943	total: 23m 50s	remaining: 2m 23s
909:	learn: 0.2051529	total: 23m 51s	remaining: 2m 21s
910:	learn: 0.2050314	total: 23m 53s	remaining: 2m 20s
911:	learn: 0.2049767	total: 23m 54s	remaining: 2m 18s
912:	learn: 0.2048457	total: 23m 56s	remaining: 2m 16s
913:	learn: 0.2048044	total: 23m 57s	remaining: 2m 15s
914:	learn

### Выводы:

1. Рассмотрены модели логистической регрессии и бустер Catboost, модели обучены и получены удовлетворяющие условию проекта (f1 > 0.75) как на кроссвалидации, так и на тестовой выборке.
2. Логистическая регрессия на 0.8% лучше модели Catboost по метрике f1. Разница на мой взгляд небольшая, но, так как скорость обучения модели логистической регрессии гораздо выше, я бы выбрал эту модель.

## 2.4 Обучение моделей на полном наборе данных и сравнение значений F1 score

Объединим обучающие и тестовые признаки и целевой признак:

In [74]:
X = vstack([X_train_vec, X_test_vec])
y = pd.concat([y_train, y_test])

Без этого объединения будет не правильно делать выводы о подобраных гиперпараметрах, так как размерность признаков изменится

Обучим модели и посмотрим на значение метрики f1

**Логистическая регрессия**

In [77]:
best_mod_log = grid_log.best_estimator_

In [79]:
def f1_calc(model):
    print(f'Значение метрики f1 => {cross_val_score(model, X, y, cv=3, scoring="f1").mean()}')

In [85]:
%%time
f1_calc(best_mod_log)

Значение метрики f1 => 0.7618427627189299
Wall time: 8.7 s


**Модель Catboost**

In [81]:
best_mod_cat = grid_cat.best_estimator_

In [82]:
f1_calc(best_mod_cat)

Learning rate set to 0.075579
0:	learn: 0.6561934	total: 1.44s	remaining: 23m 55s
1:	learn: 0.6293006	total: 2.87s	remaining: 23m 50s
2:	learn: 0.6072333	total: 4.17s	remaining: 23m 5s
3:	learn: 0.5901452	total: 5.52s	remaining: 22m 54s
4:	learn: 0.5763682	total: 6.83s	remaining: 22m 38s
5:	learn: 0.5657718	total: 8.11s	remaining: 22m 23s
6:	learn: 0.5568693	total: 9.43s	remaining: 22m 17s
7:	learn: 0.5509795	total: 10.7s	remaining: 22m 12s
8:	learn: 0.5435811	total: 12s	remaining: 22m 6s
9:	learn: 0.5375687	total: 13.4s	remaining: 22m 9s
10:	learn: 0.5313564	total: 14.7s	remaining: 22m 2s
11:	learn: 0.5234747	total: 16s	remaining: 21m 58s
12:	learn: 0.5181725	total: 17.3s	remaining: 21m 54s
13:	learn: 0.5144778	total: 18.6s	remaining: 21m 51s
14:	learn: 0.5100040	total: 19.9s	remaining: 21m 49s
15:	learn: 0.5063130	total: 21.3s	remaining: 21m 49s
16:	learn: 0.5026765	total: 22.6s	remaining: 21m 45s
17:	learn: 0.4987745	total: 23.9s	remaining: 21m 41s
18:	learn: 0.4950614	total: 25.2s	

153:	learn: 0.3479922	total: 3m 17s	remaining: 18m 7s
154:	learn: 0.3475065	total: 3m 19s	remaining: 18m 6s
155:	learn: 0.3469380	total: 3m 20s	remaining: 18m 4s
156:	learn: 0.3465135	total: 3m 21s	remaining: 18m 3s
157:	learn: 0.3461090	total: 3m 22s	remaining: 18m 1s
158:	learn: 0.3456383	total: 3m 24s	remaining: 18m
159:	learn: 0.3452068	total: 3m 25s	remaining: 17m 58s
160:	learn: 0.3447149	total: 3m 26s	remaining: 17m 57s
161:	learn: 0.3441968	total: 3m 28s	remaining: 17m 55s
162:	learn: 0.3437640	total: 3m 29s	remaining: 17m 54s
163:	learn: 0.3431210	total: 3m 30s	remaining: 17m 53s
164:	learn: 0.3426367	total: 3m 31s	remaining: 17m 51s
165:	learn: 0.3422944	total: 3m 33s	remaining: 17m 50s
166:	learn: 0.3418351	total: 3m 34s	remaining: 17m 48s
167:	learn: 0.3410051	total: 3m 35s	remaining: 17m 47s
168:	learn: 0.3405302	total: 3m 36s	remaining: 17m 46s
169:	learn: 0.3391178	total: 3m 38s	remaining: 17m 44s
170:	learn: 0.3387752	total: 3m 39s	remaining: 17m 43s
171:	learn: 0.33824

304:	learn: 0.2889170	total: 6m 40s	remaining: 15m 11s
305:	learn: 0.2886576	total: 6m 41s	remaining: 15m 10s
306:	learn: 0.2884693	total: 6m 42s	remaining: 15m 9s
307:	learn: 0.2882330	total: 6m 44s	remaining: 15m 7s
308:	learn: 0.2879906	total: 6m 45s	remaining: 15m 6s
309:	learn: 0.2877295	total: 6m 47s	remaining: 15m 6s
310:	learn: 0.2875278	total: 6m 48s	remaining: 15m 4s
311:	learn: 0.2872606	total: 6m 49s	remaining: 15m 3s
312:	learn: 0.2869931	total: 6m 51s	remaining: 15m 2s
313:	learn: 0.2866784	total: 6m 52s	remaining: 15m 1s
314:	learn: 0.2864974	total: 6m 53s	remaining: 14m 59s
315:	learn: 0.2862903	total: 6m 55s	remaining: 14m 58s
316:	learn: 0.2860435	total: 6m 56s	remaining: 14m 57s
317:	learn: 0.2858398	total: 6m 57s	remaining: 14m 55s
318:	learn: 0.2855131	total: 6m 59s	remaining: 14m 54s
319:	learn: 0.2852845	total: 7m	remaining: 14m 53s
320:	learn: 0.2850679	total: 7m 1s	remaining: 14m 51s
321:	learn: 0.2848558	total: 7m 2s	remaining: 14m 50s
322:	learn: 0.2846325	to

455:	learn: 0.2566063	total: 10m 1s	remaining: 11m 57s
456:	learn: 0.2563867	total: 10m 2s	remaining: 11m 55s
457:	learn: 0.2562311	total: 10m 3s	remaining: 11m 54s
458:	learn: 0.2560865	total: 10m 4s	remaining: 11m 53s
459:	learn: 0.2559794	total: 10m 6s	remaining: 11m 51s
460:	learn: 0.2558382	total: 10m 7s	remaining: 11m 50s
461:	learn: 0.2556588	total: 10m 8s	remaining: 11m 49s
462:	learn: 0.2554695	total: 10m 10s	remaining: 11m 47s
463:	learn: 0.2553147	total: 10m 11s	remaining: 11m 46s
464:	learn: 0.2551848	total: 10m 12s	remaining: 11m 44s
465:	learn: 0.2549805	total: 10m 13s	remaining: 11m 43s
466:	learn: 0.2547779	total: 10m 15s	remaining: 11m 42s
467:	learn: 0.2545973	total: 10m 16s	remaining: 11m 40s
468:	learn: 0.2544593	total: 10m 17s	remaining: 11m 39s
469:	learn: 0.2542234	total: 10m 19s	remaining: 11m 38s
470:	learn: 0.2540222	total: 10m 20s	remaining: 11m 36s
471:	learn: 0.2537838	total: 10m 21s	remaining: 11m 35s
472:	learn: 0.2534150	total: 10m 22s	remaining: 11m 33s

604:	learn: 0.2338154	total: 13m 14s	remaining: 8m 38s
605:	learn: 0.2337393	total: 13m 16s	remaining: 8m 37s
606:	learn: 0.2334367	total: 13m 17s	remaining: 8m 36s
607:	learn: 0.2332671	total: 13m 19s	remaining: 8m 35s
608:	learn: 0.2332070	total: 13m 21s	remaining: 8m 34s
609:	learn: 0.2330495	total: 13m 22s	remaining: 8m 33s
610:	learn: 0.2329455	total: 13m 24s	remaining: 8m 31s
611:	learn: 0.2328779	total: 13m 25s	remaining: 8m 30s
612:	learn: 0.2327271	total: 13m 26s	remaining: 8m 29s
613:	learn: 0.2326579	total: 13m 28s	remaining: 8m 28s
614:	learn: 0.2325054	total: 13m 29s	remaining: 8m 26s
615:	learn: 0.2323740	total: 13m 30s	remaining: 8m 25s
616:	learn: 0.2321846	total: 13m 31s	remaining: 8m 24s
617:	learn: 0.2320392	total: 13m 33s	remaining: 8m 22s
618:	learn: 0.2318962	total: 13m 34s	remaining: 8m 21s
619:	learn: 0.2318124	total: 13m 35s	remaining: 8m 20s
620:	learn: 0.2316765	total: 13m 37s	remaining: 8m 18s
621:	learn: 0.2315333	total: 13m 38s	remaining: 8m 17s
622:	learn

...

268:	learn: 0.3020919	total: 6m 1s	remaining: 16m 23s
269:	learn: 0.3018038	total: 6m 3s	remaining: 16m 21s
270:	learn: 0.3014109	total: 6m 4s	remaining: 16m 20s
271:	learn: 0.3009584	total: 6m 5s	remaining: 16m 19s
272:	learn: 0.3007255	total: 6m 7s	remaining: 16m 17s
273:	learn: 0.3004075	total: 6m 8s	remaining: 16m 16s
274:	learn: 0.3001724	total: 6m 9s	remaining: 16m 14s
275:	learn: 0.2999565	total: 6m 11s	remaining: 16m 13s
276:	learn: 0.2996543	total: 6m 12s	remaining: 16m 12s
277:	learn: 0.2994254	total: 6m 14s	remaining: 16m 11s
278:	learn: 0.2990810	total: 6m 15s	remaining: 16m 10s
279:	learn: 0.2988449	total: 6m 16s	remaining: 16m 8s
280:	learn: 0.2984647	total: 6m 18s	remaining: 16m 7s
281:	learn: 0.2981986	total: 6m 19s	remaining: 16m 6s
282:	learn: 0.2979308	total: 6m 20s	remaining: 16m 4s
283:	learn: 0.2976760	total: 6m 22s	remaining: 16m 3s
284:	learn: 0.2974472	total: 6m 23s	remaining: 16m 1s
285:	learn: 0.2971651	total: 6m 24s	remaining: 16m
286:	learn: 0.2968745	total

419:	learn: 0.2659072	total: 9m 23s	remaining: 12m 57s
420:	learn: 0.2657900	total: 9m 24s	remaining: 12m 56s
421:	learn: 0.2656165	total: 9m 25s	remaining: 12m 55s
422:	learn: 0.2654811	total: 9m 27s	remaining: 12m 53s
423:	learn: 0.2652710	total: 9m 28s	remaining: 12m 52s
424:	learn: 0.2650842	total: 9m 29s	remaining: 12m 51s
425:	learn: 0.2649716	total: 9m 31s	remaining: 12m 49s
426:	learn: 0.2647422	total: 9m 32s	remaining: 12m 48s
427:	learn: 0.2645043	total: 9m 33s	remaining: 12m 46s
428:	learn: 0.2643356	total: 9m 35s	remaining: 12m 45s
429:	learn: 0.2640610	total: 9m 36s	remaining: 12m 44s
430:	learn: 0.2639073	total: 9m 37s	remaining: 12m 42s
431:	learn: 0.2637254	total: 9m 39s	remaining: 12m 41s
432:	learn: 0.2635704	total: 9m 40s	remaining: 12m 40s
433:	learn: 0.2633873	total: 9m 41s	remaining: 12m 38s
434:	learn: 0.2632873	total: 9m 43s	remaining: 12m 37s
435:	learn: 0.2630026	total: 9m 44s	remaining: 12m 35s
436:	learn: 0.2628330	total: 9m 45s	remaining: 12m 34s
437:	learn

568:	learn: 0.2419428	total: 12m 41s	remaining: 9m 37s
569:	learn: 0.2418010	total: 12m 43s	remaining: 9m 35s
570:	learn: 0.2416042	total: 12m 44s	remaining: 9m 34s
571:	learn: 0.2415419	total: 12m 45s	remaining: 9m 33s
572:	learn: 0.2414298	total: 12m 47s	remaining: 9m 31s
573:	learn: 0.2412238	total: 12m 48s	remaining: 9m 30s
574:	learn: 0.2410880	total: 12m 49s	remaining: 9m 29s
575:	learn: 0.2409555	total: 12m 51s	remaining: 9m 27s
576:	learn: 0.2408215	total: 12m 52s	remaining: 9m 26s
577:	learn: 0.2406763	total: 12m 53s	remaining: 9m 24s
578:	learn: 0.2405361	total: 12m 55s	remaining: 9m 23s
579:	learn: 0.2404555	total: 12m 56s	remaining: 9m 22s
580:	learn: 0.2403286	total: 12m 57s	remaining: 9m 20s
581:	learn: 0.2401378	total: 12m 59s	remaining: 9m 19s
582:	learn: 0.2399945	total: 13m	remaining: 9m 18s
583:	learn: 0.2398690	total: 13m 1s	remaining: 9m 16s
584:	learn: 0.2397406	total: 13m 2s	remaining: 9m 15s
585:	learn: 0.2396073	total: 13m 4s	remaining: 9m 14s
586:	learn: 0.239

718:	learn: 0.2233259	total: 16m	remaining: 6m 15s
719:	learn: 0.2232740	total: 16m 2s	remaining: 6m 14s
720:	learn: 0.2230171	total: 16m 3s	remaining: 6m 12s
721:	learn: 0.2229032	total: 16m 4s	remaining: 6m 11s
722:	learn: 0.2227856	total: 16m 6s	remaining: 6m 10s
723:	learn: 0.2226364	total: 16m 7s	remaining: 6m 8s
724:	learn: 0.2225704	total: 16m 8s	remaining: 6m 7s
725:	learn: 0.2224385	total: 16m 10s	remaining: 6m 6s
726:	learn: 0.2223004	total: 16m 11s	remaining: 6m 4s
727:	learn: 0.2222119	total: 16m 13s	remaining: 6m 3s
728:	learn: 0.2220602	total: 16m 14s	remaining: 6m 2s
729:	learn: 0.2219977	total: 16m 15s	remaining: 6m
730:	learn: 0.2219244	total: 16m 17s	remaining: 5m 59s
731:	learn: 0.2218081	total: 16m 18s	remaining: 5m 58s
732:	learn: 0.2216945	total: 16m 19s	remaining: 5m 56s
733:	learn: 0.2216437	total: 16m 21s	remaining: 5m 55s
734:	learn: 0.2215176	total: 16m 22s	remaining: 5m 54s
735:	learn: 0.2213112	total: 16m 23s	remaining: 5m 52s
736:	learn: 0.2211931	total: 1

869:	learn: 0.2085241	total: 19m 22s	remaining: 2m 53s
870:	learn: 0.2084446	total: 19m 23s	remaining: 2m 52s
871:	learn: 0.2084005	total: 19m 25s	remaining: 2m 51s
872:	learn: 0.2083036	total: 19m 26s	remaining: 2m 49s
873:	learn: 0.2082065	total: 19m 27s	remaining: 2m 48s
874:	learn: 0.2081626	total: 19m 29s	remaining: 2m 47s
875:	learn: 0.2080517	total: 19m 30s	remaining: 2m 45s
876:	learn: 0.2080077	total: 19m 31s	remaining: 2m 44s
877:	learn: 0.2079098	total: 19m 32s	remaining: 2m 42s
878:	learn: 0.2078203	total: 19m 34s	remaining: 2m 41s
879:	learn: 0.2077250	total: 19m 35s	remaining: 2m 40s
880:	learn: 0.2076271	total: 19m 36s	remaining: 2m 38s
881:	learn: 0.2075833	total: 19m 38s	remaining: 2m 37s
882:	learn: 0.2074739	total: 19m 39s	remaining: 2m 36s
883:	learn: 0.2074305	total: 19m 40s	remaining: 2m 34s
884:	learn: 0.2072699	total: 19m 42s	remaining: 2m 33s
885:	learn: 0.2071722	total: 19m 43s	remaining: 2m 32s
886:	learn: 0.2071295	total: 19m 44s	remaining: 2m 30s
887:	learn

### Выводы:

В проекте были обучены модели классификации коментариев на позитивные и негативные, и были получены следующие результаты:
1. Модель логистической регрессии имеет значение метрики F1 = 0.762 на всем наборе данных.
2. Модель CatBoost имеет значение метрики F1 = 0.757 на всем наборе данных.

Для достижения этих показателей классы были взвешены, так как был сильный дисбаланс в целевом признаке.

Обе модели удовлетворяют условиям проекта (F1 должна быть не менее 0.75), но модель логистической регрессии по метрике F1 лучше модели Catboost на 0.5% и скорость обучения модели логистической регрессии выше в 450 (!) раз.

Для решения задачи классификации текстов я бы выбрал модель логистической регрессии.